# Vamos utilizar o mlflow para dar track à experiência deste notebook

A 1º coisa que têm que fazer é efetivamente começar uma **experiência**.

Cada experiência vai ter um conjunto de **runs** associadas a ela.

Devem agrupar na mesma experiência todas as runs que querem comparar.

Mesmo utilizando modelos diferentes, caso sejam modelos que corram nos mesmo dados então será util de os comparar pelo que eles devem viver na mesma experiência.

Também quando têm mais dados para treinar, para o mesmo problema, é útil utilizar a mesma experiência para comparar com os resultados anteriores.

Referências:
* [mlflow concepts](https://www.mlflow.org/docs/2.7.1/concepts.html)
* [mlflow tracking overview](https://docs.databricks.com/applications/mlflow/tracking.html)

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [3]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "Outcome"

## Mudar a diretoria onde as experiências são guardadas

A 1a coisa que vamos fazer é mudar onde as experiências vão ser guardadas.

Por defeito o mlflow vai guardar as experiências em `./mlruns` mas nós queremos que as experiências fiquem guardadas na root do projeto e não dentro da diretoria dos notebooks e portanto vamos alterar o `tracking_uri` para a pasta `../../mlruns/local`. 

**Nota:** Vamos colocar em `../../mlruns/local` em vez de `../../mlruns` porque vamos utilizar no próximo notebook um `tracking_uri` diferente que irá guardar os artifacts em `../../mlruns` pelo que se também colocarmos o tracking_uri deste notebook para essa pasta vão existir conflictos na leitura das experiências devido a estes diferentes tracking URIs possuirem formatos diferentes.

Referências:
* [mlflow tracking concepts](https://mlflow.org/docs/2.7.1/tracking.html#concepts)
* [mlflow.set_tracking_uri](https://mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.set_tracking_uri)

In [4]:
mlflow.set_tracking_uri("../../mlruns/local")

## Criar ou reutilizar uma experiência

Tendo especificado onde queremos guardar as coisas é altura de criar a experiência !

Para tal vamos utilizar o `mlflow.create_experiment` e dar lhe um nome, neste caso será "Diabetes Prediction Experiment".

Os nomes das experiências são únicos, pelo que se tentarem correr este comando uma segunda vez ele vai falhar por esse motivo

Em alturas em que temos já a experiência criada (e por isso não vamos usar o comando acima de create) e só a queremos utilizar essa experiência, então basta nos utilizar o comando `mlflow.set_experiment` e dar lhe o nome da experiência que queremos usar. O mlflow automaticamente vai assumir essa experiência para os seguintes comandos.

**Nota:** Este `mlflow.set_experiment` é na realidade um "set or create" já que se a experiência não existir ele vai cria la

Referências:
* [mlflow.create_experiment()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.create_experiment)
* [mlflow.set_experiment()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.set_experiment)

In [5]:
from pathlib import Path

Path("../../mlruns/local").mkdir(parents=True, exist_ok=True)
mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")

2023/11/27 17:35:56 INFO mlflow.tracking.fluent: Experiment with name 'Diabetes Prediction Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/random_forest/../../mlruns/local/234761152711065781'), creation_time=1701106556496, experiment_id='234761152711065781', last_update_time=1701106556496, lifecycle_stage='active', name='Diabetes Prediction Experiment', tags={}>

## Criar os datasets

A data api do mlflow ajuda a associar a uma experiência os datasets utilizados nas diferentes runs dessa experiência. Por exemplo, para se poder comparar diferentes modelos (runs) é necessário que o conjunto de teste seja o mesmo entre as diferentes runs, e a data api do mlflow permite associar às diferentes runs os datasets associados a uma experiência.

É possivel construir um Dataset de mlflow a partir de um dataframe de pandas utilizando o `mlflow.data.from_pandas()` (ver documentação associada nas referências).

Referências:
* [mlflow data api](https://mlflow.org/docs/2.7.1/python_api/mlflow.data.html)
    * [mlflow.data.from_pandas()](https://mlflow.org/docs/2.7.1/python_api/mlflow.data.html#mlflow.data.from_pandas)

In [6]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

In [7]:
train_dataset = mlflow.data.from_pandas(train_set, source=train_path, targets=TARGET_COL, name="Diabetes Train Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source=test_path, targets=TARGET_COL, name="Diabetes Test Dataset")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.ap

## Criar uma run

Para criarem uma nova run na vossa experiment basta utilizarem o comando `mlflow.start_run`. Podem dar um nome à run, como vai ser demonstrado em baixo.

**Nota:** Desde o momento em que criam esta run até ao momento em que chamam o `mlflow.end_run` esta run vai estar ativa. O que significa que não vão conseguir chamar novamente o comando de `mlflow.start_run` até fecharem primeiro a run que está ativa chamando o `mlflow.end_run` 

Referências:
* [mlflow.start_run()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.start_run)
* [mlflow.end_run()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.end_run)

In [8]:
run = mlflow.start_run(run_name="Random Forests Run")

O `start_run` vai vos retornar um objecto de `Run` onde podem consultar os metadados (como a run_id, a experiment_id, etc) associados à run fazendo `run.info`

In [9]:
run.info

<RunInfo: artifact_uri=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/random_forest/../../mlruns/local/234761152711065781/86d08183e82145709d77a3bdaad3cfde/artifacts'), end_time=None, experiment_id='234761152711065781', lifecycle_stage='active', run_id='86d08183e82145709d77a3bdaad3cfde', run_name='Random Forests Run', run_uuid='86d08183e82145709d77a3bdaad3cfde', start_time=1701106556762, status='RUNNING', user_id='gilso'>

In [10]:
RUN_ID = run.info.run_uuid

Também podem consultar os dados (métricas, parametros e tags) guardados na run fazendo `run.data`.

Como esta run acabou de ser criada ainda não terá métricas nem parametros associados a ela.

In [11]:
RUN_ID

'86d08183e82145709d77a3bdaad3cfde'

In [12]:
run.data

<RunData: metrics={}, params={}, tags={'mlflow.runName': 'Random Forests Run',
 'mlflow.source.name': 'c:\\Users\\gilso\\anaconda3\\lib\\site-packages\\ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'gilso'}>

## Guardar datasets, artefactos, métricas e parametros da run

Cada run que começam vai ter informação valiosa de se guardar, para depois ser possível ver o que se passou na run e qual foi a perfomance da mesma. Para guardar estes dados o mlflow tem 4 tipos diferentes de conceitos fundamentais:

* `mflow.log_input`: guarda um dataset da run. Podem guardar o dataset de treino e teste. Para tal têm que tirar partido da data api do mlflow (documentação nas referências).
* `mflow.log_artifact`: guarda um artefacto da run. Isto implica guardar dados associados à run. É aqui que podem guardar outros dados (que não o dataset) que estejam guardados em ficheiros e que queiram associar à vossa run.
* `mlflow.log_metric`: guarda uma métrica associada à run. É utilizando este comando que irão guardar as métricas associadas à performance do modelo associado a esta run.
* `mlfow.log_param`: guarda um parâmetro associado à run. É utilizado para guardar parâmetros que permitam perceber melhor e reproduzir o que se passou na run. Por exemplo, neste caso, a `seed` e o `test_size` são parametros que são essenciais guardarmos já que influenciam totalmente o desfecho desta run

Além destes 3 conceitos fundamentais que permitem guardar informação da run, o mlflow também permite guardar **modelos**.
Como nesta PGDS lidamos com modelos construidos com o sklearn, vamos usar o `mlflow.sklearn.log_model` para guardar os nossos modelos.

Referências:
* [mlflow logging data to runs](https://www.mlflow.org/docs/2.7.1/tracking.html#logging-data-to-runs)
    * [mlflow.log_input()](https://mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.log_input)
    * [mflow.log_artifact()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.log_artifact)
    * [mflow.log_artifacts()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.log_artifacts)
    * [mlflow.log_metric()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.log_metric)
    * [mlflow.log_metrics()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.log_metrics)
    * [mlflow.log_param()](https://www.mlflow.org/docs/2.7.1python_api/mlflow.html#mlflow.log_param)
    * [mlflow.log_params()](https://www.mlflow.org/docs/2.7.1python_api/mlflow.html#mlflow.log_params)
    * [mlflow.log_image()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.html#mlflow.log_image)
* [mflow model api](https://www.mlflow.org/docs/2.7.1/models.html#model-api)
    * [mlflow.sklearn.log_model()](https://www.mlflow.org/docs/2.7.1/python_api/mlflow.sklearn.html#mlflow.sklearn.log_model)

In [13]:
# Guardamos a seed utilizado como parametro

mlflow.log_param("seed", SEED)

42

In [14]:
# Neste ponto guardarmos o dataset de treino e de teste associado à run

mlflow.log_input(train_dataset, context="train")
mlflow.log_input(test_dataset, context="test")

c:\Users\gilso\anaconda3\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [15]:
X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

In [16]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,3,173,78,39,185,33.8,0.970,31
1,6,134,80,37,370,46.2,0.238,46
2,5,104,74,0,0,28.8,0.153,48
3,1,139,46,19,83,28.7,0.654,22
4,5,137,108,0,0,48.8,0.227,37


In [17]:
scaler = StandardScaler()

features_names = X_train.columns

X_train[features_names] = scaler.fit_transform(X_train)
X_test[features_names] = scaler.transform(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [18]:
# Vamos querer guardar também o scaler que usamos já que é uma parte vital do pre processamento de dados
# e sem ele não conseguimos reproduzir os resultados
# Como o StandardScaler é um modelo de sklearn vamos usar o mlflow.sklearn.log_model para o guardar

mlflow.sklearn.log_model(scaler, artifact_path="std_scaler")

2023/11/27 17:35:56 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
c:\Users\gilso\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [19]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.275837,1.501997,0.424428,1.171912,0.818842,0.202551,1.388551,-0.256564
1,0.597861,0.280560,0.525791,1.045640,2.323197,1.783641,-0.696657,0.975594
2,0.306628,-0.659008,0.221703,-1.290397,-0.685512,-0.434985,-0.938791,1.139881
3,-0.858303,0.437154,-1.197374,-0.090810,-0.010585,-0.447736,0.488379,-0.995858
4,0.306628,0.374516,1.944867,-1.290397,-0.685512,2.115160,-0.727992,0.236299


In [20]:
rf = RandomForestClassifier(random_state = SEED).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [21]:
# Aqui vamos guardar também a nossa Random forest, depois de treinada
# De notar que só faz sentido de a guardar depois de esta estar treinada

mlflow.sklearn.log_model(rf, artifact_path="random_forest")

In [22]:
y_preds = rf.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [23]:
accuracy_score(y_test, y_preds)

0.7368421052631579

In [24]:
feature_imp = pd.Series(rf.feature_importances_, index = X_train.columns).sort_values(ascending = False)
feature_imp

Glucose                     0.258462
BMI                         0.180059
Age                         0.144415
DiabetesPedigreeFunction    0.106236
BloodPressure               0.090174
Pregnancies                 0.078640
SkinThickness               0.072315
Insulin                     0.069700
dtype: float64

In [25]:
# A feature importance não é essencial de guardar, já que a conseguimos reproduzir, 
# mas fica mais fácil de ter logo acesso a estes dados se os tivermos guardado como artefactos da run,
# assim evita fazer nos correr novamente a run para percebermos a feature importance

feature_imp_path = "../../data/feature_importance.csv"
feature_imp.to_csv(feature_imp_path)
mlflow.log_artifact(feature_imp_path)

In [26]:
rf = RandomForestClassifier(random_state = SEED)

parameters = {'n_estimators': [10, 100, 300]}

clf = GridSearchCV(rf, parameters, cv = 5).fit(X_train, y_train)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version

In [27]:
# Vamos querer guardar também o nosso grid search (mais um modelo do sklearn)

mlflow.sklearn.log_model(clf, artifact_path="grid_search_cv")

In [28]:
tunned_rf = clf.best_estimator_

In [29]:
# E claro, vamos querer guardar também a melhor versão do nosso modelo, a nossa random forest depois do hyper parameter tunning

mlflow.sklearn.log_model(tunned_rf, "tunned_rf")

In [30]:
tunned_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [31]:
y_preds = tunned_rf.predict(X_test)

c:\Users\gilso\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [32]:
acc = accuracy_score(y_test, y_preds)

In [33]:
# Por fim vamos querer associar uma métrica de performance ao nosso modelo, neste caso a accuracy

mlflow.log_metric("accuracy", acc)

In [34]:
mlflow.end_run()

## Consultar uma run já concluida

Também é possivel consultarmos uma run que executamos previamente.

Para tal deve ser utilizado o comando `get_run` e deve ser fornecido a `run_id` associada à run que queremos reutilizar.

**Nota:** Este `get_run` não irá fazer com que a run que vão buscar fique ativa - a run ativa continua a ser a que criaram com o `start_run` - podem confirmar isto no `status` presente nos metadados (no output do `run.info`) da run que vão buscar em baixo.

In [35]:
run = mlflow.get_run(run_id=RUN_ID)

In [36]:
run.info

<RunInfo: artifact_uri=('file:///c:/Users/gilso/OneDrive/Área de '
 'Trabalho/rumos/notebooks/random_forest/../../mlruns/local/234761152711065781/86d08183e82145709d77a3bdaad3cfde/artifacts'), end_time=1701106573833, experiment_id='234761152711065781', lifecycle_stage='active', run_id='86d08183e82145709d77a3bdaad3cfde', run_name='Random Forests Run', run_uuid='86d08183e82145709d77a3bdaad3cfde', start_time=1701106556762, status='FINISHED', user_id='gilso'>

In [37]:
run.data

<RunData: metrics={'accuracy': 0.7456140350877193}, params={'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "86d08183e82145709d77a3bdaad3cfde", '
                             '"artifact_path": "std_scaler", '
                             '"utc_time_created": "2023-11-27 '
                             '17:35:56.952096", "flavors": {"sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"7fdcada833b947549ad93fad1687c7f3", '
                             '"mlflow_version": "2.8.1", "model_size_bytes": '
                             '833}, {"run_id": '
                             '"86d08183e82145709d77a3bdaad3cfde", '
                             '"artifact_path": "random_forest", '
                             '"utc_time_created": "202

### Reutilizar uma run já concluida

Caso quisessemos reutilizar (e não apenas consultar) uma run já realizada teriamos que fazer

```
mlflow.start_run(run_id=RUN_ID) 
```

Com o comando acima iriamos reactivar a run com o id presente na variável `RUN_ID`

**Nota:** Caso outra run já esteja ativa o comando acima iria falhar, já que primeiro seria necessário fazer o `mlflow.end_run` da run activa. 

## Opcional - Ver a experiência na UI do mlflow

A UI do mlflow permite ver de forma visual todas as experiências criadas e permite por exemplo, comparar, filtar e ordenar, as runs dentro de uma experiência de forma visual.

Para correr a UI do mflow é necessário executar, na Anaconda Prompt na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri ./mlruns/local`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`localhost:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Experiments` podem explorar as experiências e runs que criaram.

Referências:
* [Tracking UI](https://www.mlflow.org/docs/2.7.1/tracking.html#tracking-ui)
* [command line mlflow ui](https://www.mlflow.org/docs/2.7.1/cli.html#mlflow-ui)